In [1]:
from PIL import Image
import h5py
import anubisPlotUtils as anPlot
import json
import numpy as np
import os
import hist as hi
import matplotlib
import matplotlib.pyplot as plt
from itertools import combinations
import matplotlib.colors as colors
matplotlib.use('TkAgg')  # or 'Qt5Agg', 'GTK3Agg', etc.
import mplhep as hep
hep.style.use([hep.style.ATLAS])
import sys
import AnalysisToolAnubis as AT
import seaborn as sns
import pandas as pd
import tkinter as tk
from tkinter import ttk
from pandasgui import show
from matplotlib.patches import Circle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import convolve2d
from scipy.ndimage import label, find_objects
from scipy.spatial.distance import pdist, squareform
from matplotlib.backends.backend_pdf import PdfPages
import ipywidgets as widgets
from IPython.display import display
import sys
from scipy.stats import gaussian_kde
from matplotlib.ticker import MaxNLocator



ERROR:root:Cannot activate multiple GUI eventloops


In [2]:
#Importing the data
thisData = AT.importDatafile('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubisData\\threeHourRun_24_3_1.h5')


In [3]:
mapping = {
    0: {
        (0, 31): ('rpc0', 'eta'),
        (32, 63): ('rpc0', 'phi1'),
        (64, 95): ('rpc0', 'phi2'),
        (96, 127): ('rpc1', 'eta'),
    },
    1: {
        (0, 31): ('rpc1', 'phi1'),
        (32, 63): ('rpc1', 'phi2'),
        (64, 95): ('rpc2', 'eta'),
        (96, 127): ('rpc2', 'phi1'),
    },
    2: {
        (0, 31): ('rpc2', 'phi2'),
        (32, 63): ('rpc3', 'eta'),
        (64, 95): ('rpc3', 'phi1'),
        (96, 127): ('rpc3', 'phi2'),
    },
    3: {
        (0, 31): ('rpc4', 'eta'),
        (32, 63): ('rpc4', 'phi1'),
        (64, 95): ('rpc4', 'phi2'),
        (96, 127): ('rpc5', 'eta'),
    },
    4: {
        (0, 31): ('rpc5', 'phi1'),
        (32, 63): ('rpc5', 'phi2'),
    },
}

In [4]:
print(thisData[0][1])

[1048831, 17826047, 34603263, 51380477, 68157693, 84934909, 101712125, 118489338, 135266554, 152043767, 168820982, 185598196, 202375418, 219152635, 235929855, 252707068, 403702011, 537919724, 554696939, 571474154, 588251371, 605028586, 621805803, 638583017, 655360233, 672137448, 688914665, 705691879, 722469096, 739246312, 756023529, 772800742, 806355172, 823132391, 839909606, 856686823, 873464039, 890241255, 907018471, 923795687, 940572904, 957350122, 974127337, 990904554, 1007681771, 1024458987, 1041236203, 1074790636, 1091567853, 1108345067, 1125122284, 1141899500, 1158676716, 1175453931, 1192231148, 1209008365, 1225785580, 1242562796, 1259340014, 1276117231, 1292894446, 1309671662, 1762656501]


In [ ]:
def assign_to_coordinates(tdc, channel, mapping):
    for channel_range, (rpc, side) in mapping.get(tdc, {}).items():
        if channel_range[0] <= channel <= channel_range[1]:
            offset = channel - channel_range[0]
            if side == 'eta':
                return [rpc, 'eta', [0, offset]]
            elif side in 'phi1':
                return [rpc, 'phi', [offset, 0]]
            elif side in 'phi2':
                return[rpc, 'phi', [offset + 32, 0]]
            
    return ['Null', 'Null', [tdc, channel]]  # If no match found
def remake_data(Data, mapping):
    full_data = []
    for tdc in range(0, len(Data)):
        for events in range(0, len(Data[tdc])): 
            for hits in Data[tdc][events]:
                channel = (hits>>24)&0x7f
                coord = assign_to_coordinates(tdc, channel, mapping)
                event_time = hits&0xfffff
                
            
                
                full_data.append({
                    'tdc': tdc,
                    'event number': events,
                    'channel':channel,
                    'rpc number': coord[0],
                    'strip direction': coord[1],
                    'hit coordinate': coord[2],
                    'time': event_time
                })
    df = pd.DataFrame(full_data)
        
    return df
    
    

In [ ]:
df = remake_data(thisData, mapping)


In [20]:
show(df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [28]:
strips_on_steriod = {
    'rpc0': {'phi': [0]},
    'rpc2': {'phi': [0]},
    'rpc3': {'phi': [0]},
    'rpc4': {'eta': [31], 'phi': [0]},
    'rpc5': {'eta': [31], 'phi': [0]},
}


In [ ]:
def should_remove(row):
    rpc = row['rpc number']
    direction = row['strip direction']
    hit_coordinate = row['hit coordinate']
    
    # Check if the RPC and direction are in the strips_on_steriod dictionary
    if rpc in strips_on_steriod and direction in strips_on_steriod[rpc]:
        # Check if the hit coordinate matches the criteria for removal
        if direction == 'eta' and hit_coordinate == [0, 5]:
            return True  # Remove eta direction hits with coordinate [0, 5]
        elif direction == 'phi' and hit_coordinate == [5, 0]:
            return True  # Remove phi direction hits with coordinate [5, 0]
    
    return False

# Apply the function to determine rows to remove
mask = df.apply(should_remove, axis=1)

# Filter the DataFrame to keep rows that should not be removed
df = df[~mask]

In [24]:
show(df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [ ]:
from scipy.optimize import curve_fit
def gaussian(x, mean, amplitude, standard_deviation):
    return amplitude * np.exp(-((x - mean) ** 2) / (2 * standard_deviation ** 2))

with PdfPages('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\3HourTime.pdf') as pdf:
    for rpc_number in df['rpc number'].unique():
        df_rpc = df[df['rpc number'] == rpc_number]
        
        fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(8, 12))
        
        for i, strip_direction in enumerate(['eta', 'phi']):
            df_filtered = df_rpc[(df_rpc['strip direction'] == strip_direction) & (df_rpc['time'] >= 150) & (df_rpc['time'] <= 300)]
            time_counts = df_filtered.groupby('time').size()

            # Fit a Gaussian to the data
            bin_centres = time_counts.index
            bin_vals = time_counts.values
            
            if len(bin_centres) > 0 and len(bin_vals) > 0:
                mean_guess = np.sum(bin_centres * bin_vals) / np.sum(bin_vals)
                amplitude_guess = np.max(bin_vals)
                std_guess = np.sqrt(np.sum(bin_vals * (bin_centres - mean_guess)**2) / np.sum(bin_vals))
                
                try:
                    popt, _ = curve_fit(gaussian, bin_centres, bin_vals, p0=[mean_guess, amplitude_guess, std_guess])
                    mean, amplitude, std_dev = popt
                    
                    # Plot both the data as a line and the fit
                    axes[i].plot(bin_centres, bin_vals, '-o', label=f'{rpc_number} - {strip_direction} Data')  # Changed to '-o' for line+marker
                    axes[i].plot(bin_centres, gaussian(bin_centres, *popt), label=f'Fit (Mean: {mean:.2f}, Std Dev: {std_dev:.2f})')
                    
                except RuntimeError:
                    print(f"Could not fit a Gaussian for RPC: {rpc_number}, Direction: {strip_direction}")
                    mean, std_dev = np.nan, np.nan

                axes[i].set_xlabel('Time')
                axes[i].set_ylabel('Occurrences')
                axes[i].set_title(f'RPC: {rpc_number}, Direction: {strip_direction}')
                
                # Move the legend to avoid overlap
                axes[i].legend(loc='upper right', bbox_to_anchor=(1.3, 1))
        
        plt.tight_layout()
        pdf.savefig(fig)
        plt.close(fig)

In [6]:
#slightly different way to store data, but does the same thing, it goes like rpc - event - strip
etaHits, phiHits = AT.divideHitCountsByRPC(thisData)

In [7]:
def calculate_cluster_metrics(event, minHit=0, maxHit=20):
    cluster_sizes = []
    cluster_locations = []
    current_cluster_size = 0
    start_index = None
    
    for index, i in enumerate(event):
        if i != 0:
            current_cluster_size += i
            if start_index is None:
                start_index = index  
        elif i == 0 and current_cluster_size >= minHit and current_cluster_size < maxHit:
            if start_index is not None:           
                cluster_midpoint = (start_index + index) / 2.0 
                cluster_locations.append(cluster_midpoint)
                cluster_sizes.append(current_cluster_size)
                current_cluster_size = 0
                start_index = None 
        elif i == 0:
            current_cluster_size = 0
            start_index = None 
            
    if current_cluster_size >= minHit and current_cluster_size < maxHit and start_index is not None:
        cluster_midpoint = (start_index + len(event)) / 2.0 
        cluster_locations.append(cluster_midpoint)
        cluster_sizes.append(current_cluster_size)
    
    return cluster_locations, cluster_sizes


In [8]:
columns = ['RPC Number', 'Event Number', 'cluster_locations', 'Cluster_size']
dfcluseta = pd.DataFrame(columns=columns)
dfclusphi = pd.DataFrame(columns=columns)

for i, rpc_data in enumerate(etaHits):
    for j, event in enumerate(rpc_data):
        cluster_locations, cluster_sizes = calculate_cluster_metrics(event)
        for _ in range(len(cluster_sizes)):
            
            dfcluseta = dfcluseta.append({'RPC Number': int(i), 'Event Number': int(j), 'cluster_locations': cluster_locations[_], 'Cluster_size': int(cluster_sizes[_])}, ignore_index=True)

for i, rpc_data in enumerate(phiHits):
    for j, event in enumerate(rpc_data):
        cluster_locations, cluster_sizes = calculate_cluster_metrics(event)
        filtered_pairs = [(x, y) for x, y in zip(cluster_locations, cluster_sizes) if x >= 1]

        if filtered_pairs:
            filtered_my_list, filtered_list2 = zip(*filtered_pairs)

            cluster_locations = list(filtered_my_list)
            cluster_sizes = list(filtered_list2)
        else:

            cluster_locations = []
            cluster_sizes = []
        for _ in range(len(cluster_sizes)):
            
            dfclusphi = dfclusphi.append({'RPC Number': int(i), 'Event Number': int(j), 'cluster_locations': cluster_locations[_], 'Cluster_size': int(cluster_sizes[_])}, ignore_index=True)

C:\Users\Peter\AppData\Local\Temp\ipykernel_33908\695374553.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfcluseta = dfcluseta.append({'RPC Number': int(i), 'Event Number': int(j), 'cluster_locations': cluster_locations[_], 'Cluster_size': int(cluster_sizes[_])}, ignore_index=True)
C:\Users\Peter\AppData\Local\Temp\ipykernel_33908\695374553.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfcluseta = dfcluseta.append({'RPC Number': int(i), 'Event Number': int(j), 'cluster_locations': cluster_locations[_], 'Cluster_size': int(cluster_sizes[_])}, ignore_index=True)
C:\Users\Peter\AppData\Local\Temp\ipykernel_33908\695374553.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfcluseta = dfcluseta.append({'RPC Num

In [45]:
print(calculate_cluster_metrics(phiHits[0][0]))

([52.5], [5])


In [9]:
rpc_dfseta = {rpc: dfcluseta[dfcluseta['RPC Number'] == rpc].reset_index(drop=True) for rpc in dfcluseta['RPC Number'].unique()}
rpc_dfsphi = {rpc: dfclusphi[dfclusphi['RPC Number'] == rpc].reset_index(drop=True) for rpc in dfclusphi['RPC Number'].unique()}

def plot_rpc_histograms(rpc_dfseta, rpc_dfsphi):
    with PdfPages('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\rpc_cluster_analysisaLL.pdf') as pdf:
        for rpc in rpc_dfseta.keys():
            df_eta = rpc_dfseta[rpc]
            df_phi = rpc_dfsphi[rpc]
            for df, dataset_name in zip([df_eta, df_phi], ['eta', 'phi']):
                all_cluster_locations = []
                all_cluster_sizes = []
                
                for _, row in df.iterrows():
                    locs = row['cluster_locations']
                    sizes = row['Cluster_size']
                    
                    if not isinstance(locs, list):
                        locs = [locs] if not pd.isnull(locs) else []
                    if not isinstance(sizes, list):
                        sizes = [sizes] if not pd.isnull(sizes) else []
                    
                    all_cluster_locations.extend(locs)
                    all_cluster_sizes.extend(sizes)
                
                locations_series = pd.Series(all_cluster_locations)
                sizes_series = pd.Series(all_cluster_sizes)
                
                location_frequencies = locations_series.value_counts().sort_index()
                size_frequencies = sizes_series.value_counts().sort_index()
                
                fig, axs = plt.subplots(1, 2, figsize=(12, 6))
                fig.suptitle(f'RPC {rpc} - {dataset_name.upper()}')

                if not location_frequencies.empty:
                    axs[0].bar(location_frequencies.index, location_frequencies.values, color='skyblue', edgecolor='black')
                axs[0].set_title(f'Cluster Locations')
                axs[0].set_xlabel('Cluster Location')
                axs[0].set_ylabel('Frequency')

                if not size_frequencies.empty:
                    axs[1].bar(size_frequencies.index, size_frequencies.values, color='salmon', edgecolor='black')
                axs[1].set_title(f'Cluster Sizes')
                axs[1].set_xlabel('Cluster Size')
                axs[1].set_ylabel('Frequency')
                
                if dataset_name == 'phi':
                    axs[0].set_xlim(0, 64)
                    # axs[1].set_xlim(0, 64)
                if dataset_name == 'eta':
                    axs[0].set_xlim(0, 32)
                    # axs[1].set_xlim(0, 32)
                    
                plt.tight_layout(rect=[0, 0.03, 1, 0.95])
                pdf.savefig(fig)
                plt.close()


In [56]:
plot_rpc_histograms(rpc_dfseta, rpc_dfsphi)

In [11]:
etaTime, phiTime = AT.divideEventTimesByRPC(thisData)

In [10]:
show(df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


It would be easier to work straight from the data frame
first, we want to test how does dark counts work, by only choosing hits that are larger than 300ns

In [9]:
df_filtered300plus = df[df['time'] <= 245]

In [8]:
def dataframe_to_thisData(df):
    max_tdc = df['tdc'].max()
    max_event_number = df['event number'].max()
    
    reconstructed_data = [[[] for _ in range(max_event_number + 1)] for _ in range(max_tdc + 1)]

    for index, row in df.iterrows():
        tdc = row['tdc']
        event_number = row['event number']
        channel = row['channel']
        time = row['time']
        hits = (channel << 24) | time
        reconstructed_data[tdc][event_number].append(hits)

    return reconstructed_data

There wasn't a clear cutoff, but it is clear that below 240 and above 300 we are getting a LOT of single hits, these are ofcourse dark hits.
we should really ignore anything above 10, since that probably means when all the strips fired off at the same time

We can arbitrarily define a timing window of lets say 20 nanoseconds for each event to be considered independent to each other, this will allow me to generate a series of coincidence, and actually finding the coincidences correctly rather than arbitrarily...

In [12]:
show(df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [14]:
df = df.sort_values(by=['event number', 'rpc number', 'time'])

In [6]:
def analyze_clusters(df, window_size=5):
    all_clusters = []
    PHI_POINTS = 64
    ETA_POINTS = 32

    for event_number, event_group in df.groupby('event number'):
        for rpc, rpc_group in event_group.groupby('rpc number'):
            for direction, direction_group in rpc_group.groupby('strip direction'):
                direction_group = direction_group.copy()
                direction_group['sort_key'] = direction_group['hit coordinate'].apply(lambda coord: coord[0] if direction == 'phi' else coord[1])
                sorted_direction_group = direction_group.sort_values(by=['sort_key', 'time'])

                clusters = []
                current_cluster = []
                last_hit_time = -1

                for _, hit in sorted_direction_group.iterrows():
                    hit_point = hit['sort_key']
                    hit_time = hit['time']

                    # Start a new cluster or continue the current one
                    if not current_cluster or (
                        (hit_point - current_cluster[-1][0] <= 1 or
                        (direction == 'phi' and abs(hit_point - current_cluster[-1][0]) == PHI_POINTS - 1) or
                        (direction == 'eta' and abs(hit_point - current_cluster[-1][0]) == ETA_POINTS - 1))
                        and hit_time - last_hit_time <= window_size
                    ):
                        current_cluster.append((hit_point, hit_time))
                    else:
                        if current_cluster:
                            clusters.append(process_cluster(current_cluster))
                        current_cluster = [(hit_point, hit_time)]

                    last_hit_time = hit_time

                # Process the last cluster
                if current_cluster:
                    clusters.append(process_cluster(current_cluster))

                # Flatten the clusters into the all_clusters list
                for cluster in clusters:
                    location = (cluster['start'] + cluster['end']) / 2
                    size = cluster['size']
                    event_time = cluster['event_time']
                    all_clusters.append([event_number, rpc, direction, location, size, event_time])

    cluster_df = pd.DataFrame(all_clusters, columns=['Event Number', 'RPC Number', 'Direction', 'Cluster Location', 'Cluster Size', 'Cluster Event Time'])
    return cluster_df

def process_cluster(cluster_points):
    sorted_points = sorted(cluster_points, key=lambda x: x[0])
    start = sorted_points[0][0]
    end = sorted_points[-1][0]
    size = len(sorted_points)
    # Calculate the average event time for the cluster
    event_time = sum(point[1] for point in sorted_points) / size
    return {'start': start, 'end': end, 'size': size, 'event_time': event_time}

First, partitioning the dataset into subsets based on event number and rpc number, ensures that clusters are identified within the specific context of each event and RPC. 
Within each stratified subset, the data further segregated by strip direction(either phi or eta). By treating these directions separately, the function respects the inherent spatial distinction. 
Sequential evaluation with dynamic clustering, As the functino iterates through the sorted hits of each directionally and contextually isolated subset, it employs a dynamic clustering mechanism that builds clusters based on btoth spatial adjacency and temporal proximity. This process is sensitive to the continuity of datra points, allowing for the formation of clusters that are not only adjacent in space but also occur within a narrowly defined time window. 
The incorporation of a temporal window specified by window size introduces a nuanced layer of temporal analysis, ensuring that only hits occuring within a close temporal successino are considered part of the same lcuster. This aspect is particularly relevant in scenarios where the temporal dimension provides critical insight... probably

after dynamically assembling lcusters, the functino computes the characteristics of each cluster. 

In [15]:
Analyzeddf = analyze_clusters(df)

In [41]:
#remove noise
Analyzeddf2hitmax = Analyzeddf.loc[Analyzeddf['Cluster Size'] <= 2]
# Analyzeddf = Analyzeddf.loc[Analyzeddf['Cluster Location'] != 0]

In [44]:
show(Analyzeddf)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [7]:
show(df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [23]:
def print_progress_bar(iteration, total, prefix='', suffix='', length=50, fill='█', printEnd="\r"):
    percent = ("{0:.1f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end=printEnd)
    if iteration == total: 
        print()

If the RPC dimensions are 1 X 1.6, meaning strip separation in eta is 0.03125, and in phi is 0.025, so need to message the data.

In [33]:
def calculate_metric_for_combo(combo, rpc_separation):

    rpc_heights = {'rpc1': 1}  
    for (rpc1, rpc2), separation in rpc_separation.items():
        if rpc1 in rpc_heights and rpc2 not in rpc_heights:
            rpc_heights[rpc2] = rpc_heights[rpc1] + separation
        elif rpc2 in rpc_heights and rpc1 not in rpc_heights:
            rpc_heights[rpc1] = rpc_heights[rpc2] - separation

    locations = np.array([c['location'] for c in combo])
    uncertainties = np.array([c['uncertainty'] for c in combo])
    heights = np.array([rpc_heights[c['rpc']] for c in combo])  

    try:
        slope, intercept = np.polyfit(locations, heights, 1)
        predicted = slope * locations + intercept
        residuals = heights - predicted
        chi2 = np.sum(residuals ** 2 / heights)

    except np.linalg.LinAlgError:
        slope = 0
        intercept = 0
        chi2 = np.inf
    combined_uncertainty = np.sum(uncertainties)

    return slope, intercept, chi2, combined_uncertainty, combo

In [32]:
def analyze_inter_rpc_hit_with_timing_adjusted(df):
    paths = []
    total_events = df['Event Number'].nunique()

    adjusted_muon_speed_cm_ns = 28 

    rpc_separation = {
        ('rpc1', 'rpc2'): 0.1, # Touching
        ('rpc2', 'rpc3'): 0.1, # Touching
        ('rpc3', 'rpc4'): 60,
        ('rpc4', 'rpc5'): 60,
        ('rpc5', 'rpc6'): 0.1  # Touching
    }
    

    for event_number, event_group in df.groupby('Event Number'):
        for direction in ['eta', 'phi']:
            direction_group = event_group[event_group['Direction'] == direction]
            all_clusters = []
            
            unique_rpcs = direction_group['RPC Number'].unique()
            for rpc in unique_rpcs:
                rpc_group = direction_group[direction_group['RPC Number'] == rpc]
                for _, row in rpc_group.iterrows():
                    location_scaling = 3.125 if direction == 'eta' else 2.5
                    cluster_size_scaled = max(row['Cluster Size'] * location_scaling, location_scaling) / 2
                    all_clusters.append({
                        'rpc': rpc,
                        # 'position': No longer used directly in this simplified approach
                        'location': row['Cluster Location'] * location_scaling,
                        'event_time': row['Cluster Event Time'],
                        'uncertainty': cluster_size_scaled,
                        'original_location': row['Cluster Location']
                    })

            combination_metrics = []
            for combo in combinations(all_clusters, 3):
                if len({c['rpc'] for c in combo}) < len(combo):
                    continue
                time_diffs_are_valid = True
                for i in range(len(combo)):
                    for j in range(i+1, len(combo)):
                        rpc_pair = (combo[i]['rpc'], combo[j]['rpc'])
                        if rpc_pair not in rpc_separation and rpc_pair[::-1] not in rpc_separation:
                            continue  
                        separation_distance = rpc_separation.get(rpc_pair) or rpc_separation.get(rpc_pair[::-1])
                    
                        if separation_distance is None:

                            continue  


                        time_diff = abs(combo[i]['event_time'] - combo[j]['event_time'])
                        expected_time_diff = separation_distance / adjusted_muon_speed_cm_ns

                        intrinsic_timing_uncertainty = 5  
                        if separation_distance == 0.1:  
                            if time_diff > intrinsic_timing_uncertainty:
                                time_diffs_are_valid = False
                                break
                        else:  
                            uncertainty_margin = 5  
                            if not (time_diff <= expected_time_diff + uncertainty_margin):
                                time_diffs_are_valid = False
                                break
                    if not time_diffs_are_valid:
                        break

                if not time_diffs_are_valid:
                    continue
                if time_diffs_are_valid: 
                    metric = calculate_metric_for_combo(combo,rpc_separation) 
                    combination_metrics.append(metric) 
                

            if combination_metrics:
                best_combinations = sorted(combination_metrics, key=lambda x: x[2]) 
                selected_combination = best_combinations[0]

                paths.append({
                    'Event Number': event_number,  
                    'Direction': direction,  
                    'Slope': selected_combination[0], 
                    'Intercept': selected_combination[1], 
                    'Used Coordinates': [(c['rpc'], c['original_location'], c['uncertainty']) for c in selected_combination[-1]],
                    'chi2': selected_combination[2],
                    'Combined Uncertainty': selected_combination[3]
                })
    path_df = pd.DataFrame(paths)
    return path_df

In [34]:
temporarydf = analyze_inter_rpc_hit_with_timing_adjusted(Analyzeddf)

c:\Users\Peter\OneDrive - University of Cambridge\Desktop\Project Excel Work\PartIIIRPC\.venv\lib\site-packages\numpy\lib\polynomial.py:668: RuntimeWarning:

invalid value encountered in divide

C:\Users\Peter\AppData\Local\Temp\ipykernel_24256\1840859160.py:15: RankWarning:

Polyfit may be poorly conditioned



In [42]:
twohitmaxdf = analyze_inter_rpc_hit_with_timing_adjusted(Analyzeddf2hitmax)

c:\Users\Peter\OneDrive - University of Cambridge\Desktop\Project Excel Work\PartIIIRPC\.venv\lib\site-packages\numpy\lib\polynomial.py:668: RuntimeWarning:

invalid value encountered in divide

C:\Users\Peter\AppData\Local\Temp\ipykernel_24256\1840859160.py:15: RankWarning:

Polyfit may be poorly conditioned



In [56]:
Analyzeddf3hit = Analyzeddf.loc[Analyzeddf['Cluster Size'] == 3]
Analyzeddf4hit = Analyzeddf.loc[Analyzeddf['Cluster Size'] == 4]
Analyzeddf5hit = Analyzeddf.loc[Analyzeddf['Cluster Size'] == 5]
Analyzeddf6hit = Analyzeddf.loc[Analyzeddf['Cluster Size'] == 6]

In [57]:
df3hit = analyze_inter_rpc_hit_with_timing_adjusted(Analyzeddf3hit)
df4hit = analyze_inter_rpc_hit_with_timing_adjusted(Analyzeddf4hit)
df5hit = analyze_inter_rpc_hit_with_timing_adjusted(Analyzeddf5hit)
df6hit = analyze_inter_rpc_hit_with_timing_adjusted(Analyzeddf6hit)

C:\Users\Peter\AppData\Local\Temp\ipykernel_24256\1840859160.py:15: RankWarning:

Polyfit may be poorly conditioned

C:\Users\Peter\AppData\Local\Temp\ipykernel_24256\1840859160.py:15: RankWarning:

Polyfit may be poorly conditioned



In [58]:
df3hit.to_excel('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\reconstruct3hit.xlsx', sheet_name='Sheet1', index=False)
df4hit.to_excel('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\reconstruct4hit.xlsx', sheet_name='Sheet1', index=False)
df5hit.to_excel('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\reconstruct5hit.xlsx', sheet_name='Sheet1', index=False)
df6hit.to_excel('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\reconstruct6hit.xlsx', sheet_name='Sheet1', index=False)

In [75]:
def contains_all_rpc(items):
    rpcs = [item[0] for item in items]
    return all(rpc in rpcs for rpc in ['rpc1', 'rpc2', 'rpc3'])

temporarydfdifrpc = temporarydf[~temporarydf['Used Coordinates'].apply(contains_all_rpc)]
temporarydfdifrpc.to_excel('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\reconstructdifrpc.xlsx', sheet_name='Sheet1', index=False)
event_counts = temporarydfdifrpc['Event Number'].value_counts()
temporarydfdifrpc_filtered = temporarydfdifrpc[temporarydfdifrpc['Event Number'].isin(event_counts[event_counts > 1].index)]
temporarydfdifrpc_filtered.to_excel('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\reconstructdifrpcfiltered.xlsx', sheet_name='Sheet1', index=False)

In [77]:
show(temporarydfdifrpc_filtered)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [73]:
show(temporarydfdifrpc)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [44]:
show(twohitmaxdf)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [52]:
show(temporarydf)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [35]:
show(Analyzeddf)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [51]:
event_counts = temporarydf['Event Number'].value_counts()

temporarydf_filtered = temporarydf[temporarydf['Event Number'].isin(event_counts[event_counts > 1].index)]

In [48]:
event_counts = twohitmaxdf['Event Number'].value_counts()

twohitmaxdf_filtered = twohitmaxdf[twohitmaxdf['Event Number'].isin(event_counts[event_counts > 1].index)]

In [92]:
show(twohitmaxdf)
show(twohitmaxdfrpc)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI
PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [89]:
twohitmaxdfrpc = twohitmaxdf[~twohitmaxdf['Used Coordinates'].apply(contains_all_rpc)]
twohitmaxdfrpc.to_excel('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\reconstructdifrpc2hitmax.xlsx', sheet_name='Sheet1', index=False)
event_counts = twohitmaxdfrpc['Event Number'].value_counts()
twohitmaxdfrpc_filtered = twohitmaxdfrpc[twohitmaxdfrpc['Event Number'].isin(event_counts[event_counts > 1].index)]
twohitmaxdfrpc_filtered.to_excel('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\reconstructdifrpc2hitmaxfiltered.xlsx', sheet_name='Sheet1', index=False)

In [50]:
twohitmaxdf_filtered.to_excel('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\reconstruct2hitmaxfiltered.xlsx', sheet_name='Sheet1', index=False)

In [43]:
twohitmaxdf.to_excel('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\reconstruct2hitmax.xlsx', sheet_name='Sheet1', index=False)

In [37]:
temporarydf.to_excel('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\reconstructchi2.xlsx', sheet_name='Sheet1', index=False)

In [98]:
show(temporarydf)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [94]:
dfplots = [temporarydf, temporarydf_filtered,temporarydfdifrpc, temporarydfdifrpc_filtered, twohitmaxdf, twohitmaxdf_filtered,twohitmaxdfrpc,twohitmaxdfrpc_filtered, df3hit,df4hit,df5hit,df6hit]
dataframes = {
    'All': temporarydf,
    'All_filt': temporarydf_filtered,
    'All_rpc' : temporarydfdifrpc,
    'All_rpc_filt': temporarydfdifrpc_filtered,
    '2clus': twohitmaxdf,
    '2clus_filt': twohitmaxdf_filtered,
    '2clus_rpc': twohitmaxdfrpc,
    '2clus_rpc_filt': twohitmaxdfrpc_filtered,
    '3clus': df3hit,
    '4clus': df4hit,
    '5clus': df5hit,
    '6clus': df6hit
}
rows_count = [df.shape[0] for df in dfplots]

labels = list(dataframes.keys())

plt.figure(figsize=(30, 18))  
plt.bar(labels, rows_count, color='skyblue')
plt.xlabel('DataFrames')
plt.ylabel('successful reconstructions')
plt.title('number of successful reconstruction')
plt.savefig('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\number_of_reconstruction.pdf')

In [104]:
pdf_path = 'C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\chi2_distributions.pdf'

with PdfPages(pdf_path) as pdf:
    for label, df in dataframes.items():
        if not df.empty and 'chi2' in df.columns:
            filtered_df = df[df['chi2'] <= 0.1]
        
            if not filtered_df.empty:
                plt.figure(figsize=(10, 6))
                sns.histplot(filtered_df['chi2'], kde=True, color='skyblue', bins=50)
                plt.title(f'chi2 Distribution (<=0.1): {label}')
                plt.xlabel('chi2')
                plt.ylabel('Frequency')
                pdf.savefig()
                plt.close()

The function analyzes the linearity of particle paths between different RPCs while adjusting for timing differences.
paths: a list to store results
total_event: total number of unique events in the dataset
adjusted_muon_speed: speed o fmuons adjusted for analysis in cm/ns
rpc_separation: directionary mapping rpc pairs to their separation distance in cm

1. DataFarme grouped by Event Number, and each group is iterated over
2. For each event, the analysis is split into 'eta' and 'phi' directions. Clusters of signals within the same direction are analyzed together
3. For each RPC in the event and directions:
4. The size of each clustrer is adjusted based on its direction, turning them into cemtermeters
5. A list of all clusters is created, with each cluster represented as a directionary cointaining details such as the RPC number, cluster location, event time, uncertainty, and original location. 
6. Combinations of three clusters are created to analyze possible particle paths throught eh RPCs
7. For each combination:
8. The time difference betweent he clusters is checked against the expected time difference, calculated from the RPC separation distances and the adjusted muon speed. This step also incorporates intrinsic timing uncertainties
9. combinations with invalid timing differences are discarded
10. For valid combinations, a metric is calculated using calculate_metric_for_combo, which evalues the linearity of the paths represented byt eh cluster combination
11. Path Selection: The combination with the best metric is selected for each event and direction

Calculate_metric_for Combo:
1. A linear fit is performed with the cluster locations as the independent variable and their height as the dependent variable. This fit attempts to model the path of the particle as a straight line
2. the residuals are calculated, normalized by the uncertainties, and squared. The RSS is the sum of these squared normalized residuals and serves as the metric for the linearity fo the path
3. The sum of the uncertainties of the clusters is also calculated as part fo teh metric

BTW, RSS is a statictical measure used in regression analysis to quantify the discreancy between the observed datapoitns and the datapoitns predicted by a model. 

A small RSS indicates that the model's predictinos are closer to the actual data, so we can actually manually filter out shit as well

Roughly 166 muons per second at sea level, if it is at sea level and flat, we should expect 10,000 muons, We have only observed 576 muons, does this make sense?

What really fucking surprised me is only 1 out of every fucking shit is failing, this efficiency is way too fucking high to be true

Its pretty much 90% reconstruction rate

In [111]:
rpc_info = {
    'rpc1': {'position': 0, 'next_window': 3},
    'rpc2': {'position': 0.1, 'next_window': 3},
    'rpc3': {'position': 0.2, 'next_window': 5},
    'rpc4': {'position': 6.2, 'next_window': 5},
    'rpc5': {'position': 12.2, 'next_window': 3},
    'rpc6': {'position': 12.3, 'next_window': None}
}

for rpc, info in rpc_info.items():
    plt.plot([0, 160], [info['position'], info['position']], label=rpc)

trajectories = [
    {'event_number': 1, 'direction': 'eta', 'slope': 0.005242718446601938, 'intercept': -0.612135922330097},
    # Add more trajectories as needed
]


x_values = np.linspace(0, 160, 1000) 
for traj in trajectories:
    y_values = traj['slope'] * x_values + traj['intercept']
    plt.plot(x_values, y_values, '--', label=f"Trajectory {traj['event_number']}, {traj['direction']}")

plt.xlabel('X Position')
plt.ylabel('RPC Height')
plt.title('RPC Heights and Trajectory Lines')
plt.legend()
plt.show()

In [33]:
def plot_rpc_histograms_New(dataframe, output_file):
    with PdfPages(output_file) as pdf:
        for (rpc_number, direction), group in dataframe.groupby(['RPC Number', 'Direction']):
            fig, axs = plt.subplots(1, 3, figsize=(18, 6))
            fig.suptitle(f'RPC {rpc_number} - Direction {direction.upper()}')
            

            location_bins = np.arange(group['Cluster Location'].min(), group['Cluster Location'].max() + 2) - 0.5
            axs[0].hist(group['Cluster Location'].dropna(), bins=location_bins, color='skyblue', edgecolor='black')
            axs[0].set_title('Cluster Locations')
            axs[0].set_xlabel('Cluster Location')
            axs[0].set_ylabel('Frequency')
            axs[0].xaxis.set_major_locator(MaxNLocator(integer=True))
            

            size_bins = np.arange(group['Cluster Size'].min(), group['Cluster Size'].max() + 2) - 0.5
            axs[1].hist(group['Cluster Size'].dropna(), bins=size_bins, color='salmon', edgecolor='black')
            axs[1].set_title('Cluster Sizes')
            axs[1].set_xlabel('Cluster Size')
            axs[1].set_ylabel('Frequency')
            axs[1].xaxis.set_major_locator(MaxNLocator(integer=True))


            event_time_counts = group['Cluster Event Time'].value_counts().sort_index()
            axs[2].vlines(event_time_counts.index, 0, event_time_counts.values, colors='lightgreen', linestyles='-', lw=2)
            axs[2].set_title('Cluster Event Times')
            axs[2].set_xlabel('Cluster Event Time')
            axs[2].set_ylabel('Frequency')
            axs[2].set_ylim(bottom=0)
            axs[2].xaxis.set_major_locator(MaxNLocator(integer=True))
            
            plt.tight_layout(rect=[0, 0.03, 1, 0.95])
            pdf.savefig(fig)
            plt.close()

In [34]:
plot_rpc_histograms_New(Analyzeddf, 'C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\1D timing plot\\plot.pdf')

Also intersted in the offset of the event

In [68]:
def plot_rpc_histograms_with_offsets(dataframe, output_file):
    peak_event_times_dict = {}

    for rpc_number, group in dataframe.groupby('RPC Number'):
        event_times = group['Cluster Event Time'].dropna()
        if not event_times.empty:
            kde = gaussian_kde(event_times)
            kde_x = np.linspace(event_times.min(), event_times.max(), 500)
            kde_y = kde(kde_x)
            peak_idx = np.argmax(kde_y)
            peak_event_time = kde_x[peak_idx]
            peak_event_times_dict[rpc_number] = peak_event_time

    peak_event_times_series = pd.Series(peak_event_times_dict)

    with PdfPages(output_file) as pdf:
        fig, ax = plt.subplots(figsize=(10, 6))
        peak_event_times_series.plot(kind='bar', color='lightblue', edgecolor='black', ax=ax)
        print(peak_event_times_series)
        average_offset = peak_event_times_series.mean()
        ax.axhline(y=average_offset, color='r', linestyle='--', label=f'Average Offset: {average_offset:.2f}')

        ax.set_title('Peak Event Times by RPC with Average Offset')
        ax.set_xlabel('RPC Number')
        ax.set_ylabel('Event Time')
        ax.legend()

        plt.tight_layout()
        pdf.savefig(fig)
        plt.close(fig)

In [69]:
plot_rpc_histograms_with_offsets(Analyzeddf, 'C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\1D timing plot\\plot_rpc_histograms_with_offsets.pdf')

rpc1    255.871743
rpc2    252.418838
rpc3    255.116232
rpc4    256.787575
rpc5    247.967936
rpc6    241.180361
dtype: float64


In [39]:
dfDark = df.loc[df['time'] >=330]
AnalyzeddfDark = analyze_clusters(dfDark)
AnalyzeddfDark = AnalyzeddfDark.loc[AnalyzeddfDark['Cluster Size'] <= 15]
plot_rpc_histograms_New(AnalyzeddfDark, 'C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\1D timing plot\\plotDark.pdf')

In [37]:
show(dfDark)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [135]:
def find_coincidences(group):
    coincidences = []
    for i in range(len(group)):
        for j in range(i + 1, len(group)):

            time_i = group.iloc[i]['time']
            time_j = group.iloc[j]['time']
            coord_i = group.iloc[i]['hit coordinate']
            coord_j = group.iloc[j]['hit coordinate']
            

            if (coord_i[0] != 0 and coord_j[1] != 0 and coord_i[1] == 0 and coord_j[0] == 0) or \
               (coord_i[1] != 0 and coord_j[0] != 0 and coord_i[0] == 0 and coord_j[1] == 0):

                if abs(time_i - time_j) < 10:

                    average_time = (time_i + time_j) / 2

                    coincidence_coord = [max(coord_i[0], coord_j[0]), max(coord_i[1], coord_j[1])]
                    
                    coincidences.append({
                        'event number': group.iloc[i]['event number'],
                        'rpc number': group.iloc[i]['rpc number'],
                        'coincidence coordinate': coincidence_coord,
                        'coincidence time': average_time
                    })
    return coincidences

In [129]:
print(df.groupby(['event number', 'rpc number']))

In [132]:
coincidences = []
for _, group in df.groupby(['event number', 'rpc number']):
    coincidences.extend(find_coincidences(group))

coincidences_df = pd.DataFrame(coincidences)

In [133]:
show(coincidences_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [134]:
show(df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


Before I actually do this, I actually wanted to do something more on 1D with timing window first.... I will do that tmr

In [63]:
ni=unber

NameError: name 'unber' is not defined